##車牌辨識

請先放入predictPlate_sr、realPlate_sr兩個資料夾與haar_carplate.xml檔案後再開始執行

###開始轉換圖片尺



In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

def dirResize(src, dst):
    myfiles = glob.glob(src + '/*.jpg')  #讀取資料夾全部jpg檔案
    emptydir(dst)
    print(src + ' 資料夾：')
    print('開始轉換圖形尺寸！')
    for f in myfiles:
        fname = f.split("/")[-1]
        img = Image.open(f)
        img_new = img.resize((300, 225), PIL.Image.ANTIALIAS)  #尺寸300x225
        img_new.save(dst + '/' + fname)
    print('轉換圖形尺寸完成！\n')

import PIL
from PIL import Image
import glob
import shutil, os
from time import sleep

dirResize('realPlate_sr', 'realPlate')
dirResize('predictPlate_sr', 'predictPlate')

realPlate_sr 資料夾：
開始轉換圖形尺寸！
轉換圖形尺寸完成！

predictPlate_sr 資料夾：
開始轉換圖形尺寸！
轉換圖形尺寸完成！



###擷取車牌

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

import cv2
from PIL import Image
import glob
import shutil, os
from time import sleep

print('開始擷取車牌！')
print('無法擷取車牌的圖片：')
dstdir = 'cropPlate'
myfiles = glob.glob("realPlate/*.jpg")
emptydir(dstdir)
for imgname in myfiles:
    filename = (imgname.split('/'))[-1]  #取得檔案名稱
    img = cv2.imread(imgname)  #讀入圖形
    detector = cv2.CascadeClassifier('haar_carplate.xml')
    signs = detector.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3, minSize=(20, 20))  #框出車牌
    #割取車牌
    if len(signs) > 0 :
        for (x, y, w, h) in signs:          
            image1 = Image.open(imgname)
            image2 = image1.crop((x, y, x+w, y+h))  #擷取車牌圖形
            image3 = image2.resize((140, 40), Image.ANTIALIAS)  #轉換尺寸為140X40
            image3.save(dstdir + '/tem.jpg')
            image4 = cv2.imread(dstdir + '/tem.jpg')  #以opencv讀車牌檔
            img_gray = cv2.cvtColor(image4, cv2.COLOR_RGB2GRAY)  #灰階
            _, img_thre = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY)  #黑白
            cv2.imwrite(dstdir + '/'+ filename, img_thre)
    else:
        print(filename)

os.remove(dstdir + '/tem.jpg')  #移除暫存檔
print('擷取車牌結束！')

開始擷取車牌！
無法擷取車牌的圖片：
擷取車牌結束！


###擷取數字

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

import cv2
import glob
import shutil, os
from time import sleep

print('開始擷取車牌數字！')
emptydir('cropNum')
myfiles = glob.glob('cropPlate/*.jpg')
for f in myfiles:
    filename = (f.split('/'))[-1].replace('.jpg', '')  #移除檔名中的「.jpg」
    emptydir('cropNum/' + filename)  #以車牌號碼做資料夾名稱
    image = cv2.imread(f)  #讀取車牌號碼圖片
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #灰階
    _,thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)  #轉為黑白
    contours1 = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  #尋找輪廓
    contours = contours1[0]  #取得輪廓
    letter_image_regions = []  #文字圖形串列
    for contour in contours:  #依序處理輪廓
        (x, y, w, h) = cv2.boundingRect(contour)  #單一輪廓資料
        letter_image_regions.append((x, y, w, h))  #輪廓資料加入串列
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])  #按X坐標排序
    #存檔
    i=0
    for letter_bounding_box in letter_image_regions:  #依序處理輪廓資料
        x, y, w, h = letter_bounding_box
        if w>=5 and h>30 and h<40:  #長度>6且高度在30-48才是文字
            letter_image = gray[y:y+h, x:x+w]  #擷取圖形
            letter_image = cv2.resize(letter_image, (18, 38))
            cv2.imwrite('cropNum/' + filename + '/{}.jpg'.format(i+1), letter_image)  #存各車牌文字檔
            i += 1
print('擷取車牌數字結束！')

開始擷取車牌數字！
擷取車牌數字結束！


###建立文字庫

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

import glob
import shutil, os
from time import sleep

print('開始建立文字庫！')
emptydir('platefont')
fontlist = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']
for i in range(len(fontlist)):  #建立文字資料夾
    emptydir('platefont/' + fontlist[i])
dirs = os.listdir('cropNum')  #讀取所有檔案及資料夾
picnum = 1  #圖片記數器,讓檔名不會重複
for d in dirs:
    if os.path.isdir('cropNum/' + d):  #只處理資料夾
        myfiles = sorted(glob.glob('cropNum/' + d + '/*.jpg'))
        for i, f in enumerate(myfiles):
            shutil.copyfile(f, 'platefont/{}/{}.jpg'.format(d[i], picnum))  #存入對應資料夾
            picnum += 1
print('建立文字庫結束！')

開始建立文字庫！
建立文字庫結束！


###建立訓練資料夾

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

import cv2
import random
import glob
import shutil, os
from time import sleep

fontlist = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']
print('開始建立訓練資料！')
emptydir('data')
for n in range(len(fontlist)):
    print('產生 data/' + fontlist[n] +' 資料夾')
    emptydir('data/' + fontlist[n])
    myfiles = glob.glob('platefont/' + fontlist[n] + '/*.jpg')
    for index, f in enumerate(myfiles):
        pic_total = 500  #每個文字檔案數
        pic_each = int(pic_total / len(myfiles)) + 1
        for i in range(pic_each):  #i 為檔案名稱
            img = cv2.imread(f)
            for j in range(20):  #加入指定數量雜點
                x = random.randint(0, 17)  #以亂數設定位置
                y = random.randint(0, 37)
                cv2.circle(img, (x, y), 1, (0,0,0), -1)  #畫點
            cv2.imwrite('data/' + fontlist[n] + '/{:0>4d}.jpg'.format(index*pic_each+i+1), img)  #存檔
print('建立訓練資料結束！')

開始建立訓練資料！
產生 data/0 資料夾
產生 data/1 資料夾
產生 data/2 資料夾
產生 data/3 資料夾
產生 data/4 資料夾
產生 data/5 資料夾
產生 data/6 資料夾
產生 data/7 資料夾
產生 data/8 資料夾
產生 data/9 資料夾
產生 data/A 資料夾
產生 data/B 資料夾
產生 data/C 資料夾
產生 data/D 資料夾
產生 data/E 資料夾
產生 data/F 資料夾
產生 data/G 資料夾
產生 data/H 資料夾
產生 data/J 資料夾
產生 data/K 資料夾
產生 data/L 資料夾
產生 data/M 資料夾
產生 data/N 資料夾
產生 data/P 資料夾
產生 data/Q 資料夾
產生 data/R 資料夾
產生 data/S 資料夾
產生 data/T 資料夾
產生 data/U 資料夾
產生 data/V 資料夾
產生 data/W 資料夾
產生 data/X 資料夾
產生 data/Y 資料夾
產生 data/Z 資料夾
建立訓練資料結束！


###訓練

In [ ]:
import cv2
import os.path
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

imagedir = "data"  #訓練資料
modelname = "carplate_model.hdf5"  #模型名稱
data = []  #資料串列
labels = []  #標籤串列

#讀取資料
for image_file in paths.list_images(imagedir):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #轉為灰階
    label = image_file.split(os.path.sep)[-2]  #擷取文字資料夾名稱
    data.append(image)  #加入圖形
    labels.append(label)  #加入標籤
data = np.array(data)  #轉換為numpy array
labels = np.array(labels)

#訓練資料佔85%
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.15, random_state=0)
#標準化資料
X_train_normalize=X_train.reshape(X_train.shape[0],38,18,1).astype("float") / 255.0
X_test_normalize=X_test.reshape(X_test.shape[0],38,18,1).astype("float") / 255.0
#轉換標籤為one-hot
lb = LabelBinarizer().fit(Y_train)
Y_train_OneHot = lb.transform(Y_train)
Y_test_OneHot = lb.transform(Y_test)

#建立模型
model = Sequential()
#神經網路
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(38, 18, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(34, activation="softmax"))  #34類
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#開始訓練
model.fit(X_train_normalize, Y_train_OneHot, validation_split=0.2, batch_size=32, epochs=10, verbose=1)
model.save(modelname)  #儲存模型

#準確率
scores = model.evaluate(X_train_normalize , Y_train_OneHot)
print(scores[1])
scores2 = model.evaluate(X_test_normalize , Y_test_OneHot)
print(scores2[1])


Epoch 1/10
366/366 [==============================] - 27s 72ms/step - loss: 0.2789 - accuracy: 0.9308 - val_loss: 0.0287 - val_accuracy: 0.9935
Epoch 2/10
366/366 [==============================] - 26s 71ms/step - loss: 0.0126 - accuracy: 0.9970 - val_loss: 0.0600 - val_accuracy: 0.9843
Epoch 3/10
366/366 [==============================] - 26s 70ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.0035 - val_accuracy: 0.9990
Epoch 4/10
366/366 [==============================] - 26s 70ms/step - loss: 0.0061 - accuracy: 0.9985 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 5/10
366/366 [==============================] - 26s 70ms/step - loss: 9.7908e-05 - accuracy: 1.0000 - val_loss: 3.6078e-04 - val_accuracy: 1.0000
Epoch 6/10
366/366 [==============================] - 26s 71ms/step - loss: 4.4714e-05 - accuracy: 1.0000 - val_loss: 2.1563e-04 - val_accuracy: 1.0000
Epoch 7/10
366/366 [==============================] - 26s 71ms/step - loss: 2.5270e-05 - accuracy: 1.0000 - val_loss: 2.

###擷取文字

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

import cv2
import shutil, os
from time import sleep

emptydir('cropMono')
image = cv2.imread('cropPlate/7238N2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #灰階
_,thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)  #轉為黑白
contours1 = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  #尋找輪廓
contours = contours1[0]  #取得輪廓
letter_image_regions = []  #文字圖形串列
for contour in contours:  #依序處理輪廓
    (x, y, w, h) = cv2.boundingRect(contour)  #單一輪廓資料
    letter_image_regions.append((x, y, w, h))  #輪廓資料加入串列
letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])  #按X坐標排序
print(letter_image_regions)
#存檔
i=1
for letter_bounding_box in letter_image_regions:  #依序處理輪廓資料
    x, y, w, h = letter_bounding_box
    if w>=5 and h>30 and h<40:  #長度>6且高度在33-39才是文字
        letter_image = gray[y:y+h, x:x+w]  #擷取圖形
        letter_image = cv2.resize(letter_image, (18, 38))
        cv2.imwrite('cropMono/{}.jpg'.format(i), letter_image)  #存檔
        i += 1

[(0, 0, 14, 40), (13, 8, 18, 32), (29, 0, 12, 3), (33, 8, 16, 32), (46, 39, 1, 1), (52, 9, 15, 31), (69, 9, 16, 31), (87, 25, 3, 3), (92, 8, 15, 32), (100, 0, 9, 3), (109, 9, 15, 31), (125, 0, 15, 40), (133, 15, 1, 1)]


###預測單一車牌號

In [ ]:
from keras.models import load_model
from PIL import Image
import numpy as np
import os

labels = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']  #標籤值
datan = 0  #車牌文字數,即檔案數
for fname in os.listdir('cropMono'):
    if os.path.isfile(os.path.join('cropMono', fname)):
        datan += 1
tem_data = []
for index in range(1, (datan+1)):  #讀取預測資料
    tem_data.append((np.array(Image.open("cropMono/" + str(index) +".jpg")))/255.0)
real_data = np.stack(tem_data)  #(6,38,18)
real_data1 = np.expand_dims(real_data, axis=3)  #轉換為(6,38,18,1)
model = load_model("carplate_model.hdf5")  #讀取模型
predictions = model.predict(real_data1)  #預測資料
print('車牌號碼為：')
for i in range(len(predictions)):  #顯示結果
    print(labels[int(np.argmax(predictions[i], axis=-1))], end='') 

1/1 [==============================] - 0s 100ms/step
車牌號碼為：
7238N2

###預測車牌號

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

from keras.models import load_model
from PIL import Image
import numpy as np
import cv2
import shutil, os
from time import sleep 

labels = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']  #標籤值
#擷取車牌
imgname = '1710YC.jpg'
dirname = 'recogdata'
emptydir(dirname)
img = cv2.imread('predictPlate/' + imgname)
detector = cv2.CascadeClassifier('haar_carplate.xml')
signs = detector.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3, minSize=(20, 20))
if len(signs) > 0 :
    for (x, y, w, h) in signs:          
        image1 = Image.open('predictPlate/' + imgname)
        image2 = image1.crop((x, y, x+w, y+h))
        image3 = image2.resize((140, 40), Image.ANTIALIAS)
        image3.save('tem.jpg')
        image4 = cv2.imread('tem.jpg')
        gray = cv2.cvtColor(image4, cv2.COLOR_RGB2GRAY)
        _, img_thre = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
        cv2.imwrite('tem.jpg', img_thre)
    #分割文字
    img_tem = cv2.imread('tem.jpg')
    gray = cv2.cvtColor(img_tem, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)  #轉為黑白
    contours1 = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  #尋找輪廓
    contours = contours1[0]  #取得輪廓
    letter_image_regions = []  #文字圖形串列
    for contour in contours:  #依序處理輪廓
        (x, y, w, h) = cv2.boundingRect(contour)  #單一輪廓資料
        letter_image_regions.append((x, y, w, h))  #輪廓資料加入串列
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])  #按X坐標排序
    #存檔
    i=1
    for letter_bounding_box in letter_image_regions:  #依序處理輪廓資料
        x, y, w, h = letter_bounding_box
        if w>=5 and h>32 and h<40:  #長度>6且高度在30-48才是文字
            letter_image = gray[y:y+h, x:x+w]  #擷取圖形
            letter_image = cv2.resize(letter_image, (18, 38))
            cv2.imwrite(dirname + '/{}.jpg'.format(i), letter_image)  #存檔
            i += 1
    #辨識車牌
    datan = 0  #車牌文字數,即檔案數
    for fname in os.listdir(dirname):
        if os.path.isfile(os.path.join(dirname, fname)):
            datan += 1
    tem_data = []
    for index in range(1, (datan+1)):  #讀取預測資料
        tem_data.append((np.array(Image.open("recogdata/" + str(index) +".jpg")))/255.0)
    real_data = np.stack(tem_data)
    real_data1 = np.expand_dims(real_data, axis=3)  #(7,38,18,1)
    model = load_model("carplate_model.hdf5")  #讀取模型
    predictions = model.predict(real_data1)  #預測資料
    print('車牌號碼為：')
    for i in range(len(predictions)):  #顯示結果
        print(labels[int(np.argmax(predictions[i], axis=-1))], end='') 
else:
    print('無法擷取車牌！')
os.remove('tem.jpg')

1/1 [==============================] - 0s 70ms/step
車牌號碼為：
1710YC

###批次預測車牌號

In [ ]:
def emptydir(dirname):  #清空資料夾
    if os.path.isdir(dirname):  #資料夾存在就刪除
        shutil.rmtree(dirname)
        sleep(2)  #需延遲,否則會出錯
    os.mkdir(dirname)  #建立資料夾

from keras.models import load_model
from PIL import Image
import numpy as np
import cv2
import shutil, os
from time import sleep 
import glob

labels = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']  #標籤值
dirname = 'recogdata'
model = load_model("carplate_model.hdf5")  #讀取模型
myfiles = glob.glob('predictPlate/*.jpg')
for imgname in myfiles:
    #擷取車牌
    emptydir(dirname)
    img = cv2.imread(imgname)
    detector = cv2.CascadeClassifier('haar_carplate.xml')
    signs = detector.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3, minSize=(20, 20))
    if len(signs) > 0 :
        for (x, y, w, h) in signs:          
            image1 = Image.open(imgname)
            image2 = image1.crop((x, y, x+w, y+h))
            image3 = image2.resize((140, 40), Image.ANTIALIAS)
            image3.save('tem.jpg')
            image4 = cv2.imread('tem.jpg')
            gray = cv2.cvtColor(image4, cv2.COLOR_RGB2GRAY)
            _, img_thre = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
            cv2.imwrite('tem.jpg', img_thre)
        #分割文字
        img_tem = cv2.imread('tem.jpg')
        gray = cv2.cvtColor(img_tem, cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)  #轉為黑白
        contours1 = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  #尋找輪廓
        contours = contours1[0]  #取得輪廓
        letter_image_regions = []  #文字圖形串列
        for contour in contours:  #依序處理輪廓
            (x, y, w, h) = cv2.boundingRect(contour)  #單一輪廓資料
            letter_image_regions.append((x, y, w, h))  #輪廓資料加入串列
        letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])  #按X坐標排序
        #存檔
        i=1
        for letter_bounding_box in letter_image_regions:  #依序處理輪廓資料
            x, y, w, h = letter_bounding_box
            if w>=5 and h>32 and h<40:  #長度>6且高度在30-48才是文字
                letter_image = gray[y:y+h, x:x+w]  #擷取圖形
                letter_image = cv2.resize(letter_image, (18, 38))
                cv2.imwrite(dirname + '/{}.jpg'.format(i), letter_image)  #存檔
                i += 1
        #辨識車牌
        datan = 0  #車牌文字數,即檔案數
        for fname in os.listdir(dirname):
            if os.path.isfile(os.path.join(dirname, fname)):
                datan += 1
        tem_data = []
        for index in range(1, (datan+1)):  #讀取預測資料
            tem_data.append((np.array(Image.open("recogdata/" + str(index) +".jpg")))/255.0)
        real_data = np.stack(tem_data)
        real_data1 = np.expand_dims(real_data, axis=3)  #轉換維度符合需求->(7,38,18,1)
        predictions = model.predict(real_data1)  #預測資料
        result = []
        for i in range(len(predictions)):  #顯示結果
              result.append(labels[int(np.argmax(predictions[i], axis=-1))])
        print(imgname + ' --> ' + ''.join(result)) 
    else:
        print('無法擷取車牌！')
    ##os.remove('tem.jpg')

1/1 [==============================] - 0s 73ms/step
predictPlate/1710YC.jpg --> 1710YC
1/1 [==============================] - 0s 71ms/step
predictPlate/AXK2738.jpg --> AXK2738
1/1 [==============================] - 0s 22ms/step
predictPlate/AUN6095.jpg --> AUN6095
1/1 [==============================] - 0s 24ms/step
predictPlate/AFS0568.jpg --> AFS0568
1/1 [==============================] - 0s 25ms/step
predictPlate/EW0835.jpg --> EW0835
